In [ ]:
!pip install qiskit
import qiskit
from qiskit import IBMQ

# Load your IBM Quantum account
IBMQ.load_account()

import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile, assemble, execute
from qiskit.visualization import plot_histogram
from qiskit_ibm_runtime import QiskitRuntimeService, Options
from qiskit_ibm_provider import IBMProvider

# Get the provider
provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')

# Select the 'ibm_nairobi' device
nairobi = provider.get_backend('ibm_nairobi')

# Number of times to repeat the circuit
n = 20000

q = QuantumRegister(5, 'q')
c0 = ClassicalRegister(2, 'c0')
c2 = ClassicalRegister(1, 'c2')

# Define the circuit
bit_flip = QuantumCircuit(q, c0,c2)

# Encodes the qubit in a three-qubit entangled state  
bit_flip.cx(q[0], q[1])
bit_flip.cx(q[0], q[2])

# Adds additional two qubits for error-correction
bit_flip.cx(q[0], q[3])
bit_flip.cx(q[1], q[3])
bit_flip.cx(q[0], q[4])
bit_flip.cx(q[2], q[4])

# Measure the two additional qubits
bit_flip.measure(q[3], c0[0])
bit_flip.measure(q[4], c0[1])

# Do error correction
bit_flip.x(q[2]).c_if(c0, 1)
bit_flip.x(q[1]).c_if(c0, 2)
bit_flip.x(q[0]).c_if(c0,3)

# Decodes the qubit from the three-qubit entangled state  
bit_flip.cx(q[0], q[1])
bit_flip.cx(q[0], q[2])

# Check the state of the initial qubit
bit_flip.measure(q[0], c2[0])

# Print the quantum circuit
print("Quantum Circuit:")
print(bit_flip)

# Compile the circuit
compiled_circuit = transpile(bit_flip, nairobi)
        
# Execute the circuit on the IBM Quantum device
job = execute(bit_flip, nairobi, shots=n)

# Get the result
result = job.result()

# Get the counts for '00' and '11' outcomes
counts = result.get_counts()

# Plot the histogram
plt.hist(counts)

plt.show()

# Draw
bit_flip.draw(output='mpl')